In [13]:
import sys
sys.path.append("../../src/")
import numpy as np
import math
import uncertainpy.gradual as grad

In [14]:
# cite from： https://blog.csdn.net/weixin_42029733/article/details/89922575
from itertools import combinations

def combine(temp_list, n):
    '''根据n获得列表中的所有可能组合（n个元素为一组）'''
    temp_list2 = []
    for c in combinations(temp_list, n):
        temp_list2.append(c)
    return temp_list2

# an example
list1 = ['a', 'b', 'c']
end_list = []
for i in range(len(list1)+1):
    end_list.extend(combine(list1, i))
    # print(type(combine(list1, i)))
print(end_list)

[(), ('a',), ('b',), ('c',), ('a', 'b'), ('a', 'c'), ('b', 'c'), ('a', 'b', 'c')]


In [15]:
def compute_SHAP(bag, topic_a, cur_arg, agg_f, inf_f):
    bag = grad.BAG("../../bags/movie_recommender.bag")

    strength_values = grad.algorithms.computeStrengthValues(bag, agg_f, inf_f)

    arg_name_all = []
    # arg_name_no_topic_arg = [set()]

    topic_arg = bag.arguments[topic_a]
    for arg in bag.arguments.values():
        arg_name_all.append(arg.name)
        # print((arg.name,arg.strength))
    arg_name_no_topic_arg = arg_name_all.copy()
    arg_name_no_topic_arg.remove(topic_a)

    print(arg_name_all)
    print(arg_name_no_topic_arg)

    # remove current argument
    arg_name_no_topic_no_cur = arg_name_no_topic_arg.copy()
    arg_name_no_topic_no_cur.remove(cur_arg)
    print(arg_name_no_topic_no_cur)
    print("----------------------------")

    # compute all the combinations
    all_possible_combinations = []
    for i in range(len(arg_name_no_topic_no_cur)+1):
        all_possible_combinations.extend(combine(arg_name_no_topic_no_cur, i))
    print(f"all_possible_combinations:{all_possible_combinations}")

    # store the initial weights
    arg_initial_weight = []
    # arg_initial_weight_dict = {}
    for arg in bag.arguments.values():
        arg_initial_weight.append((arg.name,arg.initial_weight))
    arg_initial_weight_dict = dict(arg_initial_weight)
    # print(f"arg_initial_weight_dict:{arg_initial_weight_dict}")

    print("All the base score and strength in QBAF:")
    for arg in bag.arguments.values():
        print((arg.name,arg.initial_weight,arg.strength))

    coefficient = []
    difference = []
    n = len(arg_name_all)-1 # 1 is topic argument
    for i in all_possible_combinations:
        print("----------------------------")
        print(f"Combination:{i}")

        # remove arg(j) from the current combination i
        for j in i:# first remove the argument from the combination
            bag.remove_argument(j)

        strength_values = grad.algorithms.computeStrengthValues(bag, agg_f, inf_f)

        # print 1st: before removing cur arg
        print("Initial and final strength (1st):")
        for arg in bag.arguments.values():
            print((arg.name,arg.initial_weight,arg.strength))
        topic_arg = bag.arguments[topic_a]
        strength_before_drop_cur = topic_arg.strength
        print(f"strength_before_drop_cur:{strength_before_drop_cur}")

        # remove cur arg
        bag.remove_argument(cur_arg)
        strength_values = grad.algorithms.computeStrengthValues(bag, agg_f, inf_f)

        # print 2nd: after removing cur arg
        print("Initial and final strength (2nd):")
        for arg in bag.arguments.values():
            print((arg.name,arg.initial_weight,arg.strength))
        topic_arg = bag.arguments[topic_a]
        strength_after_drop_cur = topic_arg.strength
        print(f"strength_after_drop_cur:{strength_after_drop_cur}")

        # compute the diff of topic arg before and after removing
        difference.append(strength_before_drop_cur-strength_after_drop_cur)
        print(f"diff={strength_before_drop_cur-strength_after_drop_cur}")

        # compute the coefficient for SHAP value
        S = len(i)
        coefficient.append(math.factorial(n-1-S)*math.factorial(S)/math.factorial(n))
        bag = grad.BAG("../../bags/movie_recommender.bag")
        strength_values = grad.algorithms.computeStrengthValues(bag, agg_f, inf_f)
    print("----------------------------")
    print("Conclusion:")
    print(coefficient)
    print(difference)
    attribution = sum(np.multiply(coefficient, difference))
    print(attribution)
    # initializing: reset to original initial weight
    bag = grad.BAG("../../bags/movie_recommender.bag")
    strength_values = grad.algorithms.computeStrengthValues(bag, agg_f, inf_f)

    return attribution

In [16]:
# Computation via Forward Propagation
bag = grad.BAG("../../bags/movie_recommender.bag")

# # QE
# agg_f = grad.semantics.modular.SumAggregation()
# inf_f = grad.semantics.modular.QuadraticMaximumInfluence(conservativeness=1)

# DF-QUAD
agg_f = grad.semantics.modular.ProductAggregation()
inf_f = grad.semantics.modular.LinearInfluence(conservativeness=1)

# # SD-DF-QUAD
# agg_f = grad.semantics.modular.ProductAggregation()
# inf_f = grad.semantics.modular.SQDFQUADInfluence(conservativeness=1)

# # EB
# agg_f = grad.semantics.modular.SumAggregation()
# inf_f = grad.semantics.modular.EulerBasedInfluence()

# # EBT
# agg_f = grad.semantics.modular.TopAggregation()
# inf_f = grad.semantics.modular.EulerBasedInfluence()


strength_values = grad.algorithms.computeStrengthValues(bag, agg_f, inf_f)

In [17]:
# # compute attribution for one arg
# topic_a = "A"
# cur_arg = ["D"]
#
#
# attribution=compute_SHAP(bag, topic_a, cur_arg)
# print(f"attribution: \n{attribution}")

In [18]:
# compute attribution for one arg
topic_a = "m"
all_args = bag.get_arguments()
all_args_names = []
for i in all_args:
    all_args_names.append(i.get_name())
print(all_args_names)
all_args_names_without_topic = all_args_names.copy()
all_args_names_without_topic.remove(topic_a)
print(all_args_names_without_topic)
attribution = {}
for cur_arg in all_args_names_without_topic:
    attribution[cur_arg]=compute_SHAP(bag, topic_a,cur_arg, agg_f, inf_f)
print(f"attribution: \n{attribution}")

['m', 'A', '1', '2', 'D', 'W']
['A', '1', '2', 'D', 'W']
['m', 'A', '1', '2', 'D', 'W']
['A', '1', '2', 'D', 'W']
['1', '2', 'D', 'W']
----------------------------
all_possible_combinations:[(), ('1',), ('2',), ('D',), ('W',), ('1', '2'), ('1', 'D'), ('1', 'W'), ('2', 'D'), ('2', 'W'), ('D', 'W'), ('1', '2', 'D'), ('1', '2', 'W'), ('1', 'D', 'W'), ('2', 'D', 'W'), ('1', '2', 'D', 'W')]
All the base score and strength in QBAF:
('m', 0.79, 0.84774307)
('A', 0.16, 0.25786000000000003)
('1', 0.05, 0.05)
('2', 0.07, 0.07)
('D', 0.05, 0.05)
('W', 0.02, 0.02)
----------------------------
Combination:()
Initial and final strength (1st):
('m', 0.79, 0.84774307)
('A', 0.16, 0.25786000000000003)
('1', 0.05, 0.05)
('2', 0.07, 0.07)
('D', 0.05, 0.05)
('W', 0.02, 0.02)
strength_before_drop_cur:0.84774307
Initial and final strength (2nd):
('m', 0.79, 0.7963)
('1', 0.05, 0.05)
('2', 0.07, 0.07)
('D', 0.05, 0.05)
('W', 0.02, 0.02)
strength_after_drop_cur:0.7963
diff=0.051443070000000035
---------------